In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import pandas as pd
import yaml


In [ ]:
#Read main path
with open('../path_main.txt', 'r') as file:   path_main  = file.read()

dir_data_HSIchg = f'{path_main}Data/Plot_preparation/HSI_changes/'
dir_data_ThrExc = f'{path_main}Data/Plot_preparation/Threshold_Exceedance/'
dir_data_HWMId  = f'{path_main}Data/Plot_preparation/HWMId/'
dir_scripts     = f'{path_main}Scripts/'
dir_names       = f'{path_main}Scripts/Model_lists/'
dir_out         = f'{path_main}Data/EURO-CORDEX/ANOVA_input/'
if not os.path.exists(dir_out): os.mkdir(dir_out)
    

## Prepare variables and parameters

In [ ]:
#Define cities
cities = ['Lisbon', 'Madrid', 'Barcelona', 'Rome', 'Athens', 'Istanbul', 'Sofia', 'Bucharest', 'Belgrade',
          'Zagreb', 'Milan', 'Budapest', 'Munich', 'Vienna', 'Prague', 'Paris', 'Brussels', 'Amsterdam',
          'London', 'Dublin', 'Hamburg', 'Copenhagen', 'Berlin', 'Warsaw', 'Kharkiv', 'Kyiv', 'Minsk','Vilnius', 
          'Riga', 'Moscow', 'NizhnyNovgorod', 'Kazan', 'SaintPetersburg', 'Helsinki', 'Stockholm', 'Oslo']

cities = np.array(cities)

#Define HSIs
HSIs = ['TX', 'TN']

# Load city coordinates
fname_coords = dir_scripts + 'City_coordinates.yml'
with open(fname_coords, 'r') as file:
    city_coords = yaml.safe_load(file)
    
#Define scenarios and variables
RCP = 'rcp85'

#Define models and RCPs which should be used
all_models = dict()
all_models = []
with open(dir_names + 'Models_CORDEX-EUR-11_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models.append(eval(line[:-1]))

#Define warming levels
EMT_change = '3.0K' #'1.0K', '2.0K', 

N_gridcells = ''
# N_gridcells = '_3x3'


## Get data and save as CSV

In [ ]:
#Select methods to show in plot
methods     = ['HSI-changes', 'Threshold-Exceedance', 'HWMId']

#Select which time method to use for HSI changes ('JJA_mean', 'Q90', 'Ymax')
t_method = 'Ymax'
quantile = 'Q50'

#Define threshold levels
THR_levels = dict()
THR_levels['TN'] = 2 # 20 °C
THR_levels['TX'] = 3 # 30 °C

#Loop over HSIs
for i6, HSI in enumerate(HSIs):
    
    THR_level = 'Level' + str(THR_levels[HSI])
    
    met_out = dict()
    met_out['HSI-changes']          = HSI + '_DeltaChange-' + t_method
    met_out['Threshold-Exceedance'] = HSI + '_ThrExc-' + THR_level
    met_out['HWMId']                = HSI + '_HWMId-' + quantile

    #Read data for HSI change
    fname_HSIchg = dir_data_HSIchg + 'EURO-CORDEX' + N_gridcells + '/HSIs-changes_rcp85_EMT.nc'
    data_HSIchg  = xr.open_dataset(fname_HSIchg)
    data_HSIchg  = data_HSIchg.sel(city=cities, EMT_change=EMT_change, t_method=t_method)[HSI]

    #Read data for threshold exceedance
    fname_ThrExc = dir_data_ThrExc + 'EURO-CORDEX' + N_gridcells + '/HSIs-ThresholdExceedance_rcp85_EMT.nc'
    data_ThrExc  = xr.open_dataset(fname_ThrExc)
    data_ThrExc  = data_ThrExc.sel(city=cities, EMT_change=EMT_change, THR_level=THR_level, transformation='no_trans')[HSI]

    #Read data for HWMId
    fname_HWMId = dir_data_HWMId + 'EURO-CORDEX' + N_gridcells + '/HWMId-' + HSI + '_' + RCP + '_EMT.nc'
    data_HWMId  = xr.open_dataset(fname_HWMId)
    data_HWMId  = data_HWMId.sel(city=cities, EMT_change=EMT_change, quantile=quantile).HWMID

    #Save data in dict for plotting
    data_coll = dict()
    data_coll['HSI-changes']          = data_HSIchg
    data_coll['Threshold-Exceedance'] = data_ThrExc
    data_coll['HWMId']                = data_HWMId
    
    #Loop over methods
    for i5, method in enumerate(methods):

        #Collect in data frame
        data_out = pd.DataFrame(data_coll[method].values)

        #Name rows and columns
        if data_out.shape[1]==67:
            data_out.columns = ['_'.join(mod) for mod in all_models]
            data_out.index = cities
        else:
            data_out.columns = cities
            data_out.index = ['_'.join(mod) for mod in all_models]
        
        #Save as CSV
        fname_out = dir_out + 'ANOVA' + N_gridcells + '_' + met_out[method] + '_all_models.csv'
        data_out.to_csv(fname_out, index=True)    
        